# Laboratory Setup Validation

This interactive notebook provides comprehensive validation procedures to ensure your MADSci laboratory setup is functioning correctly.

## Prerequisites

Before running this notebook, ensure you have completed:
- ‚úÖ **01_service_orchestration.ipynb** - All MADSci services are running
- ‚úÖ **02_resource_templates.ipynb** - Resource templates are created
- ‚úÖ **03_initial_resources.ipynb** - Lab resources are provisioned

## Overview

Validation covers multiple layers:
- **Service connectivity and health**
- **Resource management functionality**  
- **Context propagation and ownership**
- **Workflow execution capabilities**
- **Data flow and persistence**
- **Integration testing**

## Setup and Imports

In [ ]:
# Import required modules
import sys
import time
from datetime import datetime
from pathlib import Path

try:
    # MADSci clients
    from madsci.client.data_client import DataClient
    from madsci.client.event_client import EventClient
    from madsci.client.experiment_client import ExperimentClient
    from madsci.client.resource_client import ResourceClient
    from madsci.client.workcell_client import WorkcellClient
    from madsci.common.context import MadsciContext

    # Types and utilities
    from madsci.common.types.context_types import OwnershipInfo
    from madsci.common.types.resource_types import Resource
    from madsci.common.types.resource_types.resource_enums import ResourceStatusEnum

    print("‚úÖ All MADSci modules imported successfully")

except ImportError as e:
    print(f"‚ùå Import failed: {e}")
    print("Make sure MADSci is installed and services are running")
    sys.exit(1)

In [ ]:
# Import additional libraries for validation
import builtins
import contextlib
import json
from typing import Optional

import requests

# Database connectivity (optional)
try:
    import psycopg2

    postgresql_available = True
except ImportError:
    postgresql_available = False
    print(
        "‚ÑπÔ∏è  psycopg2 not available - PostgreSQL direct connectivity tests will be skipped"
    )

try:
    import pymongo

    mongodb_available = True
except ImportError:
    mongodb_available = False
    print(
        "‚ÑπÔ∏è  pymongo not available - MongoDB direct connectivity tests will be skipped"
    )

try:
    import redis

    redis_available = True
except ImportError:
    redis_available = False
    print("‚ÑπÔ∏è  redis not available - Redis direct connectivity tests will be skipped")

print("üîß Validation tools initialized")

## Validation Test Framework

Let's set up our validation framework:

In [ ]:
from typing import Any


class ValidationFramework:
    """Framework for running and tracking validation tests."""

    def __init__(self) -> None:
        """Initialize validation framework."""
        self.results = {}
        self.start_time = datetime.now()
        self.test_context = OwnershipInfo(
            node="validation_framework",
            experiment="system_validation",
            user="test_user",
        )

    def run_test(
        self,
        test_name: str,
        test_function: callable[..., bool],
        *args: Any,
        **kwargs: Any,
    ) -> bool:
        """Run a validation test and record results."""
        print(f"\nüß™ Running: {test_name}")
        print("-" * (len(test_name) + 12))

        start_time = time.time()
        try:
            result = test_function(*args, **kwargs)
            duration = time.time() - start_time

            self.results[test_name] = {
                "status": "PASS" if result else "FAIL",
                "duration": duration,
                "timestamp": datetime.now().isoformat(),
                "result": result,
            }

            status_icon = "‚úÖ" if result else "‚ùå"
            print(
                f"{status_icon} {test_name}: {'PASSED' if result else 'FAILED'} ({duration:.2f}s)"
            )

            return result

        except Exception as e:
            duration = time.time() - start_time
            self.results[test_name] = {
                "status": "ERROR",
                "duration": duration,
                "timestamp": datetime.now().isoformat(),
                "error": str(e),
            }

            print(f"‚ùå {test_name}: ERROR - {e} ({duration:.2f}s)")
            return False

    def get_summary(self) -> dict[str, Any]:
        """Get validation summary."""
        total = len(self.results)
        passed = len([r for r in self.results.values() if r["status"] == "PASS"])
        failed = len([r for r in self.results.values() if r["status"] == "FAIL"])
        errors = len([r for r in self.results.values() if r["status"] == "ERROR"])

        return {
            "total": total,
            "passed": passed,
            "failed": failed,
            "errors": errors,
            "success_rate": (passed / total * 100) if total > 0 else 0,
            "duration": (datetime.now() - self.start_time).total_seconds(),
        }


# Initialize validation framework
validator = ValidationFramework()
print("üîç Validation framework initialized")
print(
    f"Test context: {validator.test_context.node} / {validator.test_context.experiment}"
)

## 1. Service Health Validation

First, let's validate that all MADSci services are healthy:

In [ ]:
# Service health validation
def validate_service_health() -> bool:
    """Validate that all MADSci services are healthy."""
    services = [
        ("Event Manager", 8001),
        ("Experiment Manager", 8002),
        ("Resource Manager", 8003),
        ("Data Manager", 8004),
        ("Workcell Manager", 8005),
    ]

    healthy_services = 0
    total_services = len(services)

    for service_name, port in services:
        try:
            response = requests.get(f"http://localhost:{port}/health", timeout=5)
            if response.status_code == 200:
                print(f"   ‚úÖ {service_name} (port {port}) - Healthy")
                healthy_services += 1
            else:
                print(
                    f"   ‚ùå {service_name} (port {port}) - Status {response.status_code}"
                )
        except requests.exceptions.RequestException as e:
            print(f"   ‚ùå {service_name} (port {port}) - Unreachable: {e}")

    print(
        f"\n   üìä Service Health: {healthy_services}/{total_services} services healthy"
    )
    return healthy_services == total_services


# Run service health validation
validator.run_test("Service Health Check", validate_service_health)

## 2. Database Connectivity Validation

In [ ]:
# Database connectivity validation
def validate_database_connectivity() -> bool:
    """Validate database connectivity."""
    databases_healthy = 0
    total_databases = 3

    # PostgreSQL validation
    if postgresql_available:
        try:
            # ! Don't hard code your production passwords, kids!
            conn = psycopg2.connect(
                host="localhost",
                port=5432,
                database="madsci",
                user="madsci",
                password="madsci_pass",  # noqa
            )
            cursor = conn.cursor()
            cursor.execute("SELECT 1")
            cursor.fetchone()
            conn.close()
            print("   ‚úÖ PostgreSQL - Connected")
            databases_healthy += 1
        except Exception as e:
            print(f"   ‚ùå PostgreSQL - Failed: {e}")
    else:
        print("   ‚è≠Ô∏è  PostgreSQL - Skipped (psycopg2 not available)")
        total_databases -= 1

    # MongoDB validation
    if mongodb_available:
        try:
            client = pymongo.MongoClient(
                "mongodb://localhost:27017", serverSelectionTimeoutMS=3000
            )
            client.admin.command("ping")
            print("   ‚úÖ MongoDB - Connected")
            databases_healthy += 1
        except Exception as e:
            print(f"   ‚ùå MongoDB - Failed: {e}")
    else:
        print("   ‚è≠Ô∏è  MongoDB - Skipped (pymongo not available)")
        total_databases -= 1

    # Redis validation
    if redis_available:
        try:
            r = redis.Redis(host="localhost", port=6379, decode_responses=True)
            r.ping()
            print("   ‚úÖ Redis - Connected")
            databases_healthy += 1
        except Exception as e:
            print(f"   ‚ùå Redis - Failed: {e}")
    else:
        print("   ‚è≠Ô∏è  Redis - Skipped (redis not available)")
        total_databases -= 1

    print(
        f"\n   üìä Database Health: {databases_healthy}/{total_databases} databases connected"
    )
    return databases_healthy == total_databases if total_databases > 0 else True


# Run database connectivity validation
validator.run_test("Database Connectivity", validate_database_connectivity)

## 3. Client Initialization Validation

In [ ]:
# Client initialization validation
def validate_client_initialization() -> bool:
    """Validate that all MADSci clients can be initialized."""
    clients = {}
    client_classes = [
        ("Resource", ResourceClient, {}),
        ("Event", EventClient, {}),
        ("Workcell", WorkcellClient, {}),
        ("Experiment", ExperimentClient, {}),
        ("Data", DataClient, {}),
    ]

    successful_clients = 0

    for name, client_class, kwargs in client_classes:
        try:
            client = client_class(**kwargs)
            clients[name.lower()] = client
            print(f"   ‚úÖ {name} Client - Initialized")
            successful_clients += 1
        except Exception as e:
            print(f"   ‚ùå {name} Client - Failed: {e}")

    # Store clients for later use
    validator.clients = clients

    print(
        f"\n   üìä Client Initialization: {successful_clients}/{len(client_classes)} clients initialized"
    )
    return successful_clients == len(client_classes)


# Run client initialization validation
validator.run_test("Client Initialization", validate_client_initialization)

## 4. Template Operations Validation

In [ ]:
from madsci.common.types.resource_types import Container
from madsci.common.types.resource_types.resource_enums import ContainerTypeEnum


# Template operations validation
def validate_template_operations() -> Optional[bool]:
    """Test complete template lifecycle operations."""
    if "resource" not in validator.clients:
        print("   ‚ùå Resource client not available")
        return False

    client = validator.clients["resource"]
    test_template_name = "validation_test_template"

    try:
        # Clean up any existing test template
        with contextlib.suppress(Exception):
            client.delete_template(test_template_name)

        # Test 1: Create template
        test_resource = Container(
            resource_name="ValidationPlate",
            base_type=ContainerTypeEnum.container,
            rows=8,
            columns=12,
            capacity=96,
        )

        client.create_template(
            resource=test_resource,
            template_name=test_template_name,
            description="Template for validation testing",
            required_overrides=["resource_name"],
            tags=["validation", "test"],
        )
        print("   ‚úÖ Template creation - Success")

        # Test 2: List templates
        templates = client.list_templates(tags=["validation"])
        if len(templates) >= 1:
            print("   ‚úÖ Template listing - Success")
        else:
            print("   ‚ùå Template listing - No validation templates found")
            return False

        # Test 3: Get template info
        template_info = client.get_template_info(test_template_name)
        if template_info and "description" in template_info:
            print("   ‚úÖ Template info retrieval - Success")
        else:
            print("   ‚ùå Template info retrieval - Failed")
            return False

        # Test 4: Create resource from template
        resource = client.create_resource_from_template(
            template_name=test_template_name,
            resource_name="ValidationPlate_001",
            overrides={"owner": validator.test_context.model_dump()},
            add_to_database=True,
        )
        print("   ‚úÖ Resource creation from template - Success")

        # Test 5: Clean up
        client.delete_resource(resource.resource_id)
        client.delete_template(test_template_name)
        print("   ‚úÖ Template cleanup - Success")

        return True

    except Exception as e:
        print(f"   ‚ùå Template operations failed: {e}")
        # Attempt cleanup
        with contextlib.suppress(builtins.BaseException):
            client.delete_template(test_template_name)
        return False


# Run template operations validation
validator.run_test("Template Operations", validate_template_operations)

## 5. Resource Lifecycle Validation

In [ ]:
# Resource lifecycle validation
def validate_resource_lifecycle() -> Optional[bool]:
    """Test complete resource lifecycle operations."""
    result = True
    if "resource" not in validator.clients:
        print("   ‚ùå Resource client not available")
        return False

    client = validator.clients["resource"]

    try:
        # Test 1: Create resource
        test_resource = Resource(
            resource_name="LifecycleTestResource",
            resource_class="TestResource",
            attributes={"test_value": 42},
            owner=validator.test_context,
        )

        created_resource = client.add_resource(test_resource)
        resource_id = created_resource.resource_id
        print(f"   ‚úÖ Resource creation - Success (ID: {resource_id[:8]}...)")

        # Test 2: Get resource
        retrieved_resource = client.get_resource(resource_id)
        if (
            retrieved_resource
            and retrieved_resource.resource_name == "LifecycleTestResource"
        ):
            print("   ‚úÖ Resource retrieval - Success")
        else:
            print("   ‚ùå Resource retrieval - Failed")
            result = False

        # Test 3: Update resource
        updated_resource = client.update_resource(
            resource_id,
            {
                "attributes": {"test_value": 100, "updated": True},
                "status": ResourceStatusEnum.in_use,
            },
        )
        if updated_resource.attributes.get("test_value") == 100:
            print("   ‚úÖ Resource update - Success")
        else:
            print("   ‚ùå Resource update - Failed")
            result = False

        # Test 4: List resources with filters
        resources = client.list_resources(owner_node="validation_framework")
        if any(r.resource_id == resource_id for r in resources):
            print("   ‚úÖ Resource listing/filtering - Success")
        else:
            print("   ‚ùå Resource listing/filtering - Failed")
            result = False

        # Test 5: Delete resource
        success = client.delete_resource(resource_id)
        if success:
            # Verify deletion
            deleted_resource = client.get_resource(resource_id)
            if deleted_resource is None:
                print("   ‚úÖ Resource deletion - Success")
            else:
                print("   ‚ùå Resource deletion verification - Failed")
                result = False
        else:
            print("   ‚ùå Resource deletion - Failed")
            result = False

        return result

    except Exception as e:
        print(f"   ‚ùå Resource lifecycle failed: {e}")
        return False


# Run resource lifecycle validation
validator.run_test("Resource Lifecycle", validate_resource_lifecycle)

## 6. Context Propagation Validation

In [ ]:
# Context propagation validation
def validate_context_propagation() -> bool:
    """Test context propagation across services."""
    context_tests_passed = 0
    total_context_tests = 2

    # Test 1: Event logging with context
    if "event" in validator.clients:
        try:
            event_client = validator.clients["event"]

            # Create context
            context = MadsciContext(
                ownership=validator.test_context, lab_id="01JVDFED2K18FVF0E7JM7SX09F"
            )

            event_client.log_event(
                event_type="context_validation",
                message="Testing context propagation",
                context=context,
            )
            print("   ‚úÖ Event logging with context - Success")
            context_tests_passed += 1
        except Exception as e:
            print(f"   ‚ùå Event logging with context - Failed: {e}")
    else:
        print("   ‚è≠Ô∏è  Event logging with context - Skipped (client not available)")
        total_context_tests -= 1

    # Test 2: Resource operations with context
    if "resource" in validator.clients:
        try:
            resource_client = validator.clients["resource"]

            # Create resource with context
            test_resource = Resource(
                resource_name="ContextTestResource", owner=validator.test_context
            )

            created = resource_client.add_resource(test_resource)
            if created.owner.node == validator.test_context.node:
                print("   ‚úÖ Resource creation with context - Success")
                context_tests_passed += 1

                # Clean up
                resource_client.delete_resource(created.resource_id)
            else:
                print("   ‚ùå Resource creation with context - Context not preserved")
        except Exception as e:
            print(f"   ‚ùå Resource operations with context - Failed: {e}")
    else:
        print("   ‚è≠Ô∏è  Resource operations with context - Skipped (client not available)")
        total_context_tests -= 1

    print(
        f"\n   üìä Context Propagation: {context_tests_passed}/{total_context_tests} tests passed"
    )
    return (
        context_tests_passed == total_context_tests if total_context_tests > 0 else True
    )


# Run context propagation validation
validator.run_test("Context Propagation", validate_context_propagation)

## 7. Node Communication Validation (Optional)

This test validates communication with laboratory nodes if they are running:

In [ ]:
# Node communication validation
def validate_node_communication() -> bool:
    """Test node registration and communication."""
    # Expected nodes based on example_lab configuration
    expected_nodes = [
        {"name": "liquidhandler_1", "port": 6001},
        {"name": "platereader_1", "port": 6002},
        {"name": "robotarm_1", "port": 6003},
    ]

    active_nodes = 0
    total_expected = len(expected_nodes)

    for node in expected_nodes:
        try:
            response = requests.get(
                f"http://localhost:{node['port']}/health", timeout=3
            )
            if response.status_code == 200:
                print(f"   ‚úÖ Node {node['name']} - Active")
                active_nodes += 1

                # Test actions endpoint
                try:
                    actions_response = requests.get(
                        f"http://localhost:{node['port']}/actions", timeout=3
                    )
                    if actions_response.status_code == 200:
                        actions = actions_response.json()
                        print(f"      ‚Ä¢ Actions available: {len(actions)}")
                except Exception:
                    print("      ‚Ä¢ Actions endpoint not accessible")

            else:
                print(
                    f"   ‚ùå Node {node['name']} - Unhealthy (status: {response.status_code})"
                )
        except requests.exceptions.RequestException:
            print(f"   ‚è≠Ô∏è  Node {node['name']} - Not running (optional)")

    print(f"\n   üìä Node Communication: {active_nodes}/{total_expected} nodes active")

    # Consider this test successful if at least some nodes are running or none are expected
    return True  # Nodes are optional for basic validation


# Run node communication validation
validator.run_test("Node Communication", validate_node_communication)

## 8. Workflow Execution Validation (Optional)

This test validates basic workflow submission if nodes are available:

In [ ]:
# Workflow execution validation
def validate_workflow_execution() -> Optional[bool]:
    """Test basic workflow submission and monitoring."""
    if "workcell" not in validator.clients:
        print("   ‚ùå Workcell client not available")
        return False

    client = validator.clients["workcell"]

    workflow_path = _find_example_workflow_path()
    if workflow_path is None:
        print("   ‚è≠Ô∏è  Workflow execution - Skipped (no example workflow found)")
        return True  # Skip if no workflow available

    try:
        workflow_id = _submit_workflow(client, workflow_path)
        if not workflow_id:
            print("   ‚ùå Workflow submission - No workflow ID returned")
            return False

        finished = _monitor_workflow(client, workflow_id)
        if not finished:
            print("   ‚ö†Ô∏è  Workflow execution - Timeout (workflow may still be running)")
        return True
    except Exception as e:
        print(f"   ‚ùå Workflow execution failed: {e}")
        return False


def _find_example_workflow_path() -> Optional[Path]:
    """Find the example workflow file path."""
    workflow_path = Path("../workflows/example_workflow.workflow.yaml")
    if workflow_path.exists():
        return workflow_path
    workflow_path = Path("workflows/example_workflow.workflow.yaml")
    if workflow_path.exists():
        return workflow_path
    return None


def _submit_workflow(client: WorkcellClient, workflow_path: Path) -> Optional[str]:
    """Submit the workflow and return its ID."""
    workflow_id = client.submit_workflow(
        str(workflow_path), context=validator.test_context
    )
    if workflow_id:
        print(f"   ‚úÖ Workflow submission - Success (ID: {workflow_id[:8]}...)")
        return workflow_id
    return None


def _monitor_workflow(
    client: WorkcellClient, workflow_id: str, max_wait: int = 10
) -> bool:
    """Monitor workflow status for completion."""
    start_time = time.time()
    finished = False
    while time.time() - start_time < max_wait and not finished:
        try:
            status = client.get_workflow_status(workflow_id)
            if status.state in ["completed", "failed", "cancelled"]:
                finished = True
                if status.state == "completed":
                    print("   ‚úÖ Workflow execution - Completed")
                else:
                    print(
                        f"   ‚ö†Ô∏è  Workflow execution - Finished with state: {status.state}"
                    )
            else:
                time.sleep(1)
        except Exception as e:
            print(f"   ‚ùå Workflow monitoring - Error: {e}")
            break
    return finished


# Run workflow execution validation
validator.run_test("Workflow Execution", validate_workflow_execution)

## 9. Existing Lab Resources Validation

Let's validate that resources from previous setup steps are accessible:

In [ ]:
# Existing lab resources validation


def _categorize_resources(all_resources: list[Resource]) -> dict[str, int]:
    categories = {"plates": 0, "reagents": 0, "locations": 0, "tips": 0, "other": 0}
    for resource in all_resources:
        name = resource.resource_name.lower()
        if "plate" in name and "stack" not in name:
            categories["plates"] += 1
        elif any(
            reagent in name
            for reagent in ["buffer", "dmso", "reagent", "trypsin", "media"]
        ):
            categories["reagents"] += 1
        elif any(loc in name for loc in ["deck", "hotel", "incubator", "refrigerator"]):
            categories["locations"] += 1
        elif "tip" in name:
            categories["tips"] += 1
        else:
            categories["other"] += 1
    return categories


def _display_categories(categories: dict[str, int]) -> None:
    for category, count in categories.items():
        if count > 0:
            print(f"   ‚Ä¢ {category.title()}: {count}")


def _ownership_context_stats(all_resources: list[Resource]) -> None:
    resources_with_context = len([r for r in all_resources if r.owner is not None])
    context_percentage = (
        (resources_with_context / len(all_resources)) * 100
        if len(all_resources) > 0
        else 0
    )
    print(
        f"   üë• Resources with ownership context: {resources_with_context}/{len(all_resources)} ({context_percentage:.0f}%)"
    )


def _sample_resource_validation(
    client: ResourceClient, all_resources: list[Resource]
) -> tuple[int, int]:
    sample_resources = all_resources[:3]
    valid_resources = 0
    for resource in sample_resources:
        with contextlib.suppress(Exception):
            retrieved = client.get_resource(resource.resource_id)
            if retrieved and retrieved.resource_id == resource.resource_id:
                valid_resources += 1
    print(
        f"   üîç Resource retrieval validation: {valid_resources}/{len(sample_resources)} successful"
    )
    return valid_resources, len(sample_resources)


def validate_existing_resources() -> bool:
    """Validate that lab resources from setup are accessible."""
    if "resource" not in validator.clients:
        print("   ‚ùå Resource client not available")
        return False

    client = validator.clients["resource"]

    try:
        all_resources = client.list_resources()
        print(f"   üìä Total resources found: {len(all_resources)}")

        if len(all_resources) == 0:
            print("   ‚ö†Ô∏è  No resources found - lab may need initial setup")
            return True

        templates = client.list_templates()
        print(f"   üìã Templates available: {len(templates)}")

        categories = _categorize_resources(all_resources)
        _display_categories(categories)
        _ownership_context_stats(all_resources)
        valid_resources, sample_size = _sample_resource_validation(
            client, all_resources
        )

        return len(all_resources) > 0 and (valid_resources / sample_size) >= 0.5

    except Exception as e:
        print(f"   ‚ùå Resource validation failed: {e}")
        return False


# Run existing resources validation
validator.run_test("Existing Lab Resources", validate_existing_resources)

## 10. System Integration Test

A comprehensive test that combines multiple services:

In [ ]:
# System integration test
def validate_system_integration() -> bool:
    """Comprehensive integration test across multiple services."""
    integration_steps_passed = 0
    total_integration_steps = 4

    try:
        # Step 1: Create a test experiment context
        integration_context = OwnershipInfo(
            node="integration_test",
            experiment=f"integration_test_{int(time.time())}",
            user="validation_system",
        )
        print(f"   üî¨ Test experiment: {integration_context.experiment}")
        integration_steps_passed += 1

        # Step 2: Log integration test start event
        if "event" in validator.clients:
            event_client = validator.clients["event"]
            event_client.log_event(
                event_type="integration_test_start",
                message="Starting system integration validation",
                context=MadsciContext(
                    ownership=integration_context, lab_id="01JVDFED2K18FVF0E7JM7SX09F"
                ),
            )
            print("   üìù Integration start event logged")
            integration_steps_passed += 1
        else:
            print("   ‚è≠Ô∏è  Event logging skipped (client not available)")

        # Step 3: Create and manage a test resource
        if "resource" in validator.clients:
            resource_client = validator.clients["resource"]

            # Create integration test resource
            test_resource = Resource(
                resource_name=f"IntegrationTestResource_{int(time.time())}",
                resource_class="IntegrationTest",
                attributes={
                    "test_type": "system_integration",
                    "created_at": datetime.now().isoformat(),
                },
                owner=integration_context,
                status=ResourceStatusEnum.in_use,
            )

            created_resource = resource_client.add_resource(test_resource)
            resource_id = created_resource.resource_id
            print(f"   üß™ Test resource created: {created_resource.resource_name}")

            # Update the resource
            resource_client.update_resource(
                resource_id,
                {
                    "attributes": {
                        **created_resource.attributes,
                        "integration_step": "validated",
                        "updated_at": datetime.now().isoformat(),
                    },
                    "status": ResourceStatusEnum.available,
                },
            )
            print("   üîÑ Test resource updated")
            integration_steps_passed += 1

        else:
            print("   ‚è≠Ô∏è  Resource management skipped (client not available)")

        # Step 4: Log integration test completion
        if "event" in validator.clients:
            event_client.log_event(
                event_type="integration_test_complete",
                message=f"System integration validation completed - {integration_steps_passed} steps passed",
                context=MadsciContext(
                    ownership=integration_context, lab_id="01JVDFED2K18FVF0E7JM7SX09F"
                ),
            )
            print("   ‚úÖ Integration completion event logged")
            integration_steps_passed += 1

        # Cleanup: Delete test resource
        if "resource" in validator.clients and "resource_id" in locals():
            try:
                resource_client.delete_resource(resource_id)
                print("   üßπ Test resource cleaned up")
            except Exception as e:
                print(f"   ‚ö†Ô∏è  Test resource cleanup failed: {e}")

        print(
            f"\n   üìä Integration steps: {integration_steps_passed}/{total_integration_steps} completed"
        )
        return integration_steps_passed >= (
            total_integration_steps - 1
        )  # Allow for one optional step to fail

    except Exception as e:
        print(f"   ‚ùå System integration failed: {e}")
        return False


# Run system integration validation
validator.run_test("System Integration", validate_system_integration)

## Validation Summary and Report

Let's generate a comprehensive validation report:

In [ ]:
# Generate validation summary
summary = validator.get_summary()

print("\n" + "=" * 60)
print("üîç MADSCI LABORATORY VALIDATION REPORT")
print("=" * 60)

print(f"\nüìÖ Validation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"‚è±Ô∏è  Total Duration: {summary['duration']:.2f} seconds")
print("üß™ Lab Context: validation_framework / system_validation")

print("\nüìä Test Results Summary:")
print("-" * 30)
print(f"Total Tests: {summary['total']}")
print(f"‚úÖ Passed: {summary['passed']}")
print(f"‚ùå Failed: {summary['failed']}")
print(f"üö´ Errors: {summary['errors']}")
print(f"üìà Success Rate: {summary['success_rate']:.1f}%")

print("\nüîç Detailed Test Results:")
print("-" * 35)
for test_name, result in validator.results.items():
    status_icon = {"PASS": "‚úÖ", "FAIL": "‚ùå", "ERROR": "üö´"}.get(
        result["status"], "‚ùì"
    )

    print(f"{status_icon} {test_name}: {result['status']} ({result['duration']:.2f}s)")

    if result["status"] == "ERROR" and "error" in result:
        print(f"    ‚îî‚îÄ Error: {result['error']}")

# Overall assessment
print("\nüéØ Overall Assessment:")
print("-" * 25)

if summary["success_rate"] >= 90:
    assessment = "üü¢ EXCELLENT"
    message = "Your MADSci laboratory is fully operational and ready for use!"
elif summary["success_rate"] >= 75:
    assessment = "üü° GOOD"
    message = (
        "Your MADSci laboratory is mostly operational with minor issues to address."
    )
elif summary["success_rate"] >= 50:
    assessment = "üü† NEEDS ATTENTION"
    message = "Your MADSci laboratory has significant issues that should be resolved before production use."
else:
    assessment = "üî¥ CRITICAL ISSUES"
    message = (
        "Your MADSci laboratory has critical issues that must be addressed immediately."
    )

print(f"Status: {assessment}")
print(f"Message: {message}")

# Recommendations
print("\nüí° Recommendations:")
print("-" * 20)

if summary["failed"] > 0 or summary["errors"] > 0:
    print("‚Ä¢ Review failed tests above and address underlying issues")
    print("‚Ä¢ Check service logs: docker compose logs [service-name]")
    print("‚Ä¢ Verify all prerequisites are installed and configured")

if summary["success_rate"] < 100:
    print("‚Ä¢ Consider re-running validation after addressing issues")
    print("‚Ä¢ Check the troubleshooting section in previous notebooks")

if summary["success_rate"] >= 75:
    print("‚Ä¢ Your lab setup is ready for tutorial workflows")
    print("‚Ä¢ Consider exploring the scenarios/ directory for example use cases")

print("\nüìã Next Steps:")
print("-" * 15)

if summary["success_rate"] >= 90:
    print("‚úÖ Setup Complete! You can now:")
    print("   ‚Ä¢ Run the tutorial workflows in ../tutorials/")
    print("   ‚Ä¢ Explore example scenarios in ../scenarios/")
    print("   ‚Ä¢ Start developing your own laboratory workflows")
    print("   ‚Ä¢ Access the MADSci dashboard at http://localhost:8080")
elif summary["success_rate"] >= 75:
    print("‚ö†Ô∏è  Partial Setup Complete:")
    print("   ‚Ä¢ Address the failed tests above")
    print("   ‚Ä¢ Re-run this validation notebook")
    print("   ‚Ä¢ Proceed with caution to tutorials")
else:
    print("‚ùå Setup Issues Detected:")
    print("   ‚Ä¢ Review and fix all failing tests")
    print("   ‚Ä¢ Check service connectivity and configuration")
    print("   ‚Ä¢ Consider restarting services: just down && just up")
    print("   ‚Ä¢ Re-run previous setup notebooks if needed")

print("\n" + "=" * 60)
print("Validation report complete. Thank you for using MADSci!")
print("=" * 60)

## Export Validation Report

Let's save the validation report for record keeping:

In [ ]:
# Export validation report
def export_validation_report() -> Path | None:
    """Export validation report to JSON file."""
    try:
        report_data = {
            "validation_metadata": {
                "report_date": datetime.now().isoformat(),
                "lab_id": "01JVDFED2K18FVF0E7JM7SX09F",
                "validation_framework_version": "1.0",
                "total_duration": summary["duration"],
            },
            "summary": summary,
            "test_results": validator.results,
            "assessment": {
                "overall_status": assessment.split()[1],  # Extract status level
                "message": message,
                "ready_for_production": summary["success_rate"] >= 90,
            },
            "system_info": {
                "validation_context": validator.test_context.model_dump(),
                "services_tested": [
                    "Event Manager",
                    "Experiment Manager",
                    "Resource Manager",
                    "Data Manager",
                    "Workcell Manager",
                ],
                "optional_components": ["Laboratory Nodes", "Workflow Execution"],
            },
        }

        # Save to file
        report_file = Path("../requirements/validation_report.json")
        report_file.parent.mkdir(exist_ok=True)

        with Path(report_file).open("w") as f:
            json.dump(report_data, f, indent=2, default=str)

        return report_file

    except Exception as e:
        print(f"‚ùå Failed to export validation report: {e}")
        return None


exported_report = export_validation_report()

if exported_report:
    file_size = exported_report.stat().st_size / 1024
    print(f"\n‚úÖ Validation report exported to: {exported_report}")
    print(f"   File size: {file_size:.1f} KB")
    print(f"   Success rate: {summary['success_rate']:.1f}%")
    print(f"   Tests completed: {summary['total']}")
else:
    print("\n‚ùå Failed to export validation report")

print(
    "\nüìã Validation complete. Your MADSci laboratory setup has been thoroughly tested."
)

## Summary

üéâ **Validation Complete!** 

You have successfully validated your MADSci laboratory setup across multiple dimensions:

### ‚úÖ Completed Validation Areas:
- **Service Health**: All MADSci microservices connectivity
- **Database Connectivity**: PostgreSQL, MongoDB, Redis connections
- **Client Initialization**: All MADSci Python clients
- **Template Operations**: Complete template lifecycle
- **Resource Lifecycle**: CRUD operations on lab resources
- **Context Propagation**: Ownership and tracking across services
- **Node Communication**: Laboratory instrument connectivity (optional)
- **Workflow Execution**: Basic workflow submission (optional)
- **Existing Resources**: Validation of provisioned lab resources
- **System Integration**: End-to-end multi-service operations

### üìä What Was Tested:
- Core service availability and health
- Database connectivity and persistence
- Resource management functionality
- Template system operations
- Context and ownership tracking
- Inter-service communication
- Data integrity and validation

### üîß Validation Framework Features:
- Automated test execution and timing
- Comprehensive error handling and reporting
- Detailed success/failure analysis
- JSON report export for record keeping
- Clear recommendations for next steps

---

## üéØ Setup Journey Complete!

**Congratulations!** You have completed the MADSci laboratory setup journey:

1. **‚úÖ Service Orchestration** - Microservices architecture deployed
2. **‚úÖ Resource Templates** - Reusable resource definitions created
3. **‚úÖ Initial Resources** - Laboratory inventory provisioned
4. **‚úÖ Validation** - Comprehensive testing completed (this notebook)

### üöÄ Ready for Next Steps:
- **Interactive Tutorials**: `../tutorials/01_basic_setup/`
- **Real-world Scenarios**: `../scenarios/pharmaceutical_screening/`
- **Custom Workflows**: Develop your own laboratory automation
- **Dashboard Access**: http://localhost:8080 (when available)

### üí° Pro Tips for Production Use:
- Run validation regularly to catch issues early
- Monitor service logs for performance insights
- Export and archive validation reports for compliance
- Set up automated health checks for continuous monitoring
- Scale services based on actual laboratory workload

**Your MADSci-powered laboratory is now ready for autonomous scientific discovery!** üß¨ü§ñ